In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import collections

In [43]:
df = pd.read_csv('data/train_data_raw.csv')

In [44]:
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime'],
      dtype='object')

In [45]:
df.shape

(300000, 41)

__Dataframe detail:__
- Each month (Jan - Dec) has randomly 25000 flights. 
- The data only take from 2019

### Data Cleaning

__Check the ralation between cancelled and diverted with arr_delay__

In [46]:
df[(df['cancelled'] == 1) | (df['diverted'] == 1)]['arr_delay'].isna().sum()

6554

In [47]:
# If the flight is cancelled or diverted 
# => we dont have the value for arr_delay
# => we remove the rows that the flights had beed cancelled or diverted

In [48]:
df = df[df['cancelled'] != 1]
df = df[df['diverted'] != 1]

In [49]:
df.shape

(293446, 41)

#### Table flights_test
This table consists of subset of columns from table flights. It represents flights from January 2020 which will be used for evaluation. Therefore, we are missing some features that we are not suppossed to know before the flight lands.

##### Variables:

__fl_date__: Flight Date (yyyy-mm-dd)  
__mkt_unique_carrier__: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.  
__branded_code_share__: Reporting Carrier Operated or Branded Code Share Partners  
__mkt_carrier__: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.  
__mkt_carrier_fl_num__: Flight Number  
__op_unique_carrier__: Unique Scheduled Operating Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users,for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.  
__tail_num__: Tail Number  
__op_carrier_fl_num__: Flight Number  
__origin_airport_id__: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.  
__origin__: Origin Airport  
__origin_city_name__: Origin Airport, City Name  
__dest_airport_id__: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.  
__dest__: Destination Airport  
__dest_city_name__: Destination Airport, City Name  
__crs_dep_time__: CRS Departure Time (local time: hhmm)  
__crs_arr_time__: CRS Arrival Time (local time: hhmm)  
__dup__: Duplicate flag marked Y if the flight is swapped based on Form-3A data  
__crs_elapsed_time__: CRS Elapsed Time of Flight, in Minutes  
__flights__: Number of Flights  
__distance__: Distance between airports (miles)  

#### _Discussion_
- _We only have 19 features (columns) in the test data, so we try to use the same feature for our sample data for futher exploration_
- _We add 1 more feature_ __arr_delay__ _is our target_
- I will use 'carrier_delay','weather_delay','dep_delay' to convert categorical feature into ordinal features based on weight

In [50]:
df_flight = df[['fl_date', 'mkt_unique_carrier', 'branded_code_share', 
        'mkt_carrier', 'mkt_carrier_fl_num',
        'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id',
        'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name',
        'crs_dep_time', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
        'arr_delay', 'carrier_delay','weather_delay','dep_delay', 'taxi_out', 'taxi_in']]
# 'carrier_delay','weather_delay','dep_delay' is used to convert categorical feature into ordinal features based on weight

In [51]:
df_flight_test = pd.read_csv('data/flights_test.csv')

In [52]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293446 entries, 0 to 299999
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   fl_date             293446 non-null  object 
 1   mkt_unique_carrier  293446 non-null  object 
 2   branded_code_share  293446 non-null  object 
 3   mkt_carrier         293446 non-null  object 
 4   mkt_carrier_fl_num  293446 non-null  int64  
 5   op_unique_carrier   293446 non-null  object 
 6   tail_num            293446 non-null  object 
 7   op_carrier_fl_num   293446 non-null  int64  
 8   origin_airport_id   293446 non-null  int64  
 9   origin              293446 non-null  object 
 10  origin_city_name    293446 non-null  object 
 11  dest_airport_id     293446 non-null  int64  
 12  dest                293446 non-null  object 
 13  dest_city_name      293446 non-null  object 
 14  crs_dep_time        293446 non-null  int64  
 15  crs_arr_time        293446 non-nul

In [53]:
df_flight.isna().sum()

fl_date                    0
mkt_unique_carrier         0
branded_code_share         0
mkt_carrier                0
mkt_carrier_fl_num         0
op_unique_carrier          0
tail_num                   0
op_carrier_fl_num          0
origin_airport_id          0
origin                     0
origin_city_name           0
dest_airport_id            0
dest                       0
dest_city_name             0
crs_dep_time               0
crs_arr_time               0
dup                        0
crs_elapsed_time           0
flights                    0
distance                   0
arr_delay                  0
carrier_delay         236653
weather_delay         236653
dep_delay                  0
taxi_out                   0
taxi_in                    0
dtype: int64

In [54]:
# transform the feature fl_date into datetime_type
df_flight['fl_date'] = pd.to_datetime(df_flight['fl_date'])
df_flight['fl_date'].dtypes

C:\Users\nqhuy\AppData\Local\Temp/ipykernel_18848/258140478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight['fl_date'] = pd.to_datetime(df_flight['fl_date'])


dtype('<M8[ns]')

In [55]:
df_flight_test['fl_date'] = pd.to_datetime(df_flight_test['fl_date'])
df_flight_test['fl_date'].dtypes

dtype('<M8[ns]')

__Check the relation between ['mkt_unique_carrier', 'mkt_carrier', 'branded_code_share']__

In [56]:
df_flight[['mkt_unique_carrier', 'mkt_carrier', 'branded_code_share']]

,mkt_unique_carrier,mkt_carrier,branded_code_share
0,DL,DL,DL
1,WN,WN,WN
2,AA,AA,AA
3,UA,UA,UA_CODESHARE
4,DL,DL,DL
...,...,...,...
299995,DL,DL,DL
299996,AA,AA,AA
299997,AA,AA,AA
299998,UA,UA,UA_CODESHARE


In [57]:
df[df['mkt_unique_carrier'] != df['mkt_carrier']]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime


In [58]:
df_flight.groupby(['mkt_unique_carrier','mkt_carrier']).arr_delay.count().sort_values(ascending=False)
# duplicate columns, use mkt_unique_carrier

mkt_unique_carrier  mkt_carrier
AA                  AA             76281
DL                  DL             64773
UA                  UA             56590
WN                  WN             48898
AS                  AS             16160
B6                  B6             10944
NK                  NK              7689
F9                  F9              4820
G4                  G4              3829
HA                  HA              3462
Name: arr_delay, dtype: int64

In [59]:
df_flight = df_flight.drop(columns=['mkt_carrier'])

In [60]:
df_flight_test = df_flight_test.drop(columns=['mkt_carrier'])

In [61]:
df_flight[df_flight['mkt_unique_carrier'] != df_flight['branded_code_share']]\
[['mkt_unique_carrier', 'branded_code_share']]

,mkt_unique_carrier,branded_code_share
3,UA,UA_CODESHARE
6,AA,AA_CODESHARE
8,AA,AA_CODESHARE
10,UA,UA_CODESHARE
11,AA,AA_CODESHARE
...,...,...
299987,AA,AA_CODESHARE
299989,UA,UA_CODESHARE
299993,UA,UA_CODESHARE
299994,UA,UA_CODESHARE


__Discussion:__
- We can see that there is no difference between 'mkt_unique_carrier' and 'mkt_carrier'
- The differences between 'mkt_unique_carrier' and 'branded_code_share' is only the suffix _CODESHARE  
=> So we can use only 1 feature among these 3 features:  (**'mkt_unique_carrier'**)

In [62]:
df_flight = df_flight.drop(columns=['branded_code_share'])

In [63]:
df_flight_test = df_flight_test.drop(columns=['branded_code_share'])

In [64]:
df_flight.shape

(293446, 24)

In [65]:
df_flight_test.shape

(150623, 19)

In [66]:
df_flight_test.isna().sum()

Unnamed: 0              0
fl_date                 0
mkt_unique_carrier      0
mkt_carrier_fl_num      0
op_unique_carrier       0
tail_num              124
op_carrier_fl_num       0
origin_airport_id       0
origin                  0
origin_city_name        0
dest_airport_id         0
dest                    0
dest_city_name          0
crs_dep_time            0
crs_arr_time            0
dup                     0
crs_elapsed_time        0
flights                 0
distance                0
dtype: int64

### Flight_number 
__*a) Flight_number with high incidence of weather_delay*__

In [67]:
bad_weather_airport = df_flight.groupby(['mkt_carrier_fl_num']).weather_delay.mean().round(0).sort_values(ascending=False)
bad_weather_airport

mkt_carrier_fl_num
7414    597.0
7375    568.0
4685    466.0
6283    383.0
7369    242.0
        ...  
9372      NaN
9385      NaN
9386      NaN
9387      NaN
9390      NaN
Name: weather_delay, Length: 6747, dtype: float64

In [68]:
bad_weather_dest = df_flight.groupby(['dest']).weather_delay.mean().round(0).sort_values(ascending=False)
bad_weather_dest

dest
ATY    294.0
BRD    284.0
BET    113.0
CYS     81.0
HGR     71.0
       ...  
DLG      NaN
HIB      NaN
IMT      NaN
OGD      NaN
PPG      NaN
Name: weather_delay, Length: 373, dtype: float64

In [69]:
bad_weather_airport= bad_weather_airport.reset_index()
bad_weather_airport

,mkt_carrier_fl_num,weather_delay
0,7414,597.0
1,7375,568.0
2,4685,466.0
3,6283,383.0
4,7369,242.0
...,...,...
6742,9372,NaN
6743,9385,NaN
6744,9386,NaN
6745,9387,NaN


In [70]:
bad_weather_dest= bad_weather_dest.reset_index()
bad_weather_dest

,dest,weather_delay
0,ATY,294.0
1,BRD,284.0
2,BET,113.0
3,CYS,81.0
4,HGR,71.0
...,...,...
368,DLG,NaN
369,HIB,NaN
370,IMT,NaN
371,OGD,NaN


In [71]:
bad_weather_airport.fillna(bad_weather_airport['weather_delay'].mean(),inplace= True)

In [72]:
bad_weather_dest.fillna(bad_weather_dest['weather_delay'].mean(),inplace= True)

In [73]:
bad_weather_airport.columns= ['mkt_carrier_fl_num','mkt_carrier_fl_num_weather_delay']

In [74]:
bad_weather_dest.columns= ['dest','dest_weather_delay']

In [75]:
bad_weather_airport.head()

,mkt_carrier_fl_num,mkt_carrier_fl_num_weather_delay
0,7414,597.0
1,7375,568.0
2,4685,466.0
3,6283,383.0
4,7369,242.0


In [76]:
bad_weather_dest.head()

,dest,dest_weather_delay
0,ATY,294.0
1,BRD,284.0
2,BET,113.0
3,CYS,81.0
4,HGR,71.0


__*b) mkt_carrier_fl_num with high incidence of departure_delay*__

In [77]:
departure_delay_airport= df_flight.groupby(['mkt_carrier_fl_num']).dep_delay.mean().round(0).sort_values(ascending=False)
departure_delay_airport= departure_delay_airport.reset_index()

In [78]:
departure_delay_dest= df_flight.groupby(['dest']).dep_delay.mean().round(0).sort_values(ascending=False)
departure_delay_dest= departure_delay_dest.reset_index()

In [79]:
departure_delay_airport.head()

,mkt_carrier_fl_num,dep_delay
0,9330,1111.0
1,6283,295.0
2,6717,222.0
3,6602,190.0
4,6499,146.0


In [80]:
departure_delay_dest.head()

,dest,dep_delay
0,OGS,97.0
1,HYA,58.0
2,CMX,56.0
3,ATY,52.0
4,STC,49.0


In [81]:
departure_delay_airport.columns=['mkt_carrier_fl_num','mkt_carrier_fl_num_dep_delay']

In [82]:
departure_delay_dest.columns=['dest','dest_dep_delay']

In [83]:
departure_delay_airport.head()

,mkt_carrier_fl_num,mkt_carrier_fl_num_dep_delay
0,9330,1111.0
1,6283,295.0
2,6717,222.0
3,6602,190.0
4,6499,146.0


In [84]:
departure_delay_dest.head()

,dest,dest_dep_delay
0,OGS,97.0
1,HYA,58.0
2,CMX,56.0
3,ATY,52.0
4,STC,49.0


__*c) mkt_carrier_fl_num with high incidence of arr_delay*__

In [85]:
bad_delay_airport = df_flight.groupby(['mkt_carrier_fl_num']).arr_delay.mean().round(0).sort_values(ascending=False)
bad_delay_airport

mkt_carrier_fl_num
9330    1092.0
6283     299.0
6717     233.0
6602     189.0
6499     156.0
         ...  
6701     -29.0
6530     -29.0
6714     -29.0
6901     -32.0
6699     -39.0
Name: arr_delay, Length: 6747, dtype: float64

In [86]:
bad_delay_dest = df_flight.groupby(['dest']).arr_delay.mean().round(0).sort_values(ascending=False)
bad_delay_dest

dest
OGS    99.0
HYA    54.0
CMX    54.0
ATY    49.0
STC    44.0
       ... 
HIB   -14.0
APN   -14.0
RHI   -14.0
ADK   -18.0
PPG   -18.0
Name: arr_delay, Length: 373, dtype: float64

In [87]:
bad_delay_airport = bad_delay_airport.reset_index()
bad_delay_airport

,mkt_carrier_fl_num,arr_delay
0,9330,1092.0
1,6283,299.0
2,6717,233.0
3,6602,189.0
4,6499,156.0
...,...,...
6742,6701,-29.0
6743,6530,-29.0
6744,6714,-29.0
6745,6901,-32.0


In [88]:
bad_delay_dest = bad_delay_dest.reset_index()

In [89]:
bad_delay_airport.columns= ['mkt_carrier_fl_num','mkt_carrier_fl_num_arr_delay']

In [90]:
bad_delay_dest.columns= ['dest','dest_arr_delay']

In [91]:
bad_delay_airport.head()

,mkt_carrier_fl_num,mkt_carrier_fl_num_arr_delay
0,9330,1092.0
1,6283,299.0
2,6717,233.0
3,6602,189.0
4,6499,156.0


__*d) mkt_carrier_fl_num with high incidence of taxi_out*__

In [92]:
taxi_out_airport = df_flight.groupby(['mkt_carrier_fl_num']).taxi_out.mean().round(0).sort_values(ascending=False)
taxi_out_airport

mkt_carrier_fl_num
6703    61.0
7400    59.0
9370    49.0
6554    46.0
7881    45.0
        ... 
6693     7.0
6675     6.0
6678     6.0
6501     6.0
6701     6.0
Name: taxi_out, Length: 6747, dtype: float64

In [93]:
taxi_out_airport = taxi_out_airport.reset_index()
taxi_out_airport

,mkt_carrier_fl_num,taxi_out
0,6703,61.0
1,7400,59.0
2,9370,49.0
3,6554,46.0
4,7881,45.0
...,...,...
6742,6693,7.0
6743,6675,6.0
6744,6678,6.0
6745,6501,6.0


In [94]:
taxi_out_airport.columns = ['mkt_carrier_fl_num', 'mkt_carrier_fl_num_taxi_out']
taxi_out_airport

,mkt_carrier_fl_num,mkt_carrier_fl_num_taxi_out
0,6703,61.0
1,7400,59.0
2,9370,49.0
3,6554,46.0
4,7881,45.0
...,...,...
6742,6693,7.0
6743,6675,6.0
6744,6678,6.0
6745,6501,6.0


__*e) mkt_carrier_fl_num with high incidence of taxi_in*__

In [95]:
taxi_in_airport = df_flight.groupby(['mkt_carrier_fl_num']).taxi_in.mean().round(0).sort_values(ascending=False)
taxi_in_airport

mkt_carrier_fl_num
6928    27.0
6499    26.0
6913    26.0
6460    21.0
6736    20.0
        ... 
7390     2.0
6675     2.0
6431     2.0
6439     1.0
6733     1.0
Name: taxi_in, Length: 6747, dtype: float64

In [96]:
taxi_in_dest = df_flight.groupby(['dest']).taxi_in.mean().round(0).sort_values(ascending=False)
taxi_in_dest

dest
ORD    14.0
PPG    13.0
CLT    13.0
DFW    12.0
JFK    11.0
       ... 
HIB     3.0
DUT     2.0
MKK     2.0
LNY     2.0
JHM     2.0
Name: taxi_in, Length: 373, dtype: float64

In [97]:
taxi_in_airport = taxi_in_airport.reset_index()
taxi_in_airport

,mkt_carrier_fl_num,taxi_in
0,6928,27.0
1,6499,26.0
2,6913,26.0
3,6460,21.0
4,6736,20.0
...,...,...
6742,7390,2.0
6743,6675,2.0
6744,6431,2.0
6745,6439,1.0


In [98]:
taxi_in_dest = taxi_in_dest.reset_index()

In [99]:
taxi_in_airport.columns = ['mkt_carrier_fl_num', 'mkt_carrier_fl_num_taxi_in']
taxi_in_airport

,mkt_carrier_fl_num,mkt_carrier_fl_num_taxi_in
0,6928,27.0
1,6499,26.0
2,6913,26.0
3,6460,21.0
4,6736,20.0
...,...,...
6742,7390,2.0
6743,6675,2.0
6744,6431,2.0
6745,6439,1.0


In [100]:
taxi_in_dest.columns = ['dest', 'dest_taxi_in']

In [101]:
df_flight[['origin', 'origin_airport_id', 'origin_city_name']]

,origin,origin_airport_id,origin_city_name
0,BOS,10721,"Boston, MA"
1,TPA,15304,"Tampa, FL"
2,PHL,14100,"Philadelphia, PA"
3,MHT,13296,"Manchester, NH"
4,SLC,14869,"Salt Lake City, UT"
...,...,...,...
299995,LAX,12892,"Los Angeles, CA"
299996,PHX,14107,"Phoenix, AZ"
299997,CLT,11057,"Charlotte, NC"
299998,CWA,11203,"Mosinee, WI"


In [102]:
df_flight['origin'].nunique()

373

In [103]:
df_flight['origin_airport_id'].nunique()

373

In [104]:
df_flight['origin_city_name'].nunique()

365

- We can see that the 'origin' and 'origin_airport_id' have the same meaning
- One city can have more than 1 airport
=> we can only use the feature **origin** among 3 features 

- We can also use this filter for the destination

In [105]:
df_flight = df_flight.drop(columns=['origin_airport_id', 'origin_city_name'])

In [106]:
df_flight_test = df_flight_test.drop(columns=['origin_airport_id', 'origin_city_name'])

In [107]:
df_flight.shape

(293446, 22)

In [108]:
df_flight.columns

Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier_fl_num',
       'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'arr_delay', 'carrier_delay', 'weather_delay', 'dep_delay', 'taxi_out',
       'taxi_in'],
      dtype='object')

In [109]:
df_flight = df_flight.drop(columns=['dest_airport_id', 'dest_city_name'])

In [110]:
df_flight_test.shape

(150623, 17)

In [111]:
df_flight_test = df_flight_test.drop(columns=['dest_airport_id', 'dest_city_name'])

In [112]:
# The dup column has only data 'N'
df_flight['dup'].unique()

array(['N'], dtype=object)

In [113]:
# remove the dup column
df_flight = df_flight.drop(columns=['dup'])

In [114]:
df_flight_test = df_flight_test.drop(columns=['dup'])

### mkt carrier
__a) *mkt carrier with high incidence of carrier_delay*__

In [115]:
delayed_mkt_carrier= df_flight.groupby(['mkt_unique_carrier']).carrier_delay.mean().round(0).sort_values(ascending=False)
delayed_mkt_carrier.head()

mkt_unique_carrier
B6    28.0
DL    25.0
G4    24.0
UA    22.0
AA    21.0
Name: carrier_delay, dtype: float64

In [116]:
delayed_mkt_carrier= delayed_mkt_carrier.reset_index()

In [117]:
delayed_mkt_carrier

,mkt_unique_carrier,carrier_delay
0,B6,28.0
1,DL,25.0
2,G4,24.0
3,UA,22.0
4,AA,21.0
5,HA,21.0
6,F9,18.0
7,WN,17.0
8,NK,16.0
9,AS,14.0


In [118]:
delayed_mkt_carrier.columns= ['mkt_unique_carrier','mkt_carrier_delay']

In [119]:
delayed_mkt_carrier.head()

,mkt_unique_carrier,mkt_carrier_delay
0,B6,28.0
1,DL,25.0
2,G4,24.0
3,UA,22.0
4,AA,21.0


__*b) mkt_unique_carrier with high incidence of departure delay*__

In [120]:
departure_delay_mkt_carrier= df_flight.groupby(['mkt_unique_carrier']).dep_delay.mean().round(0).sort_values(ascending=False)
departure_delay_mkt_carrier= departure_delay_mkt_carrier.reset_index()

In [121]:
departure_delay_mkt_carrier

,mkt_unique_carrier,dep_delay
0,B6,18.0
1,F9,15.0
2,UA,15.0
3,NK,11.0
4,AA,10.0
5,DL,10.0
6,G4,10.0
7,WN,10.0
8,AS,4.0
9,HA,2.0


In [122]:
departure_delay_mkt_carrier.columns=['mkt_unique_carrier','mkt_carrier_dep_delay']

In [123]:
departure_delay_mkt_carrier.head()

,mkt_unique_carrier,mkt_carrier_dep_delay
0,B6,18.0
1,F9,15.0
2,UA,15.0
3,NK,11.0
4,AA,10.0


In [124]:
# very high number of difference between mkt_unique_carrier and op_unique_carrier
df_flight[df_flight['mkt_unique_carrier'] != df_flight['op_unique_carrier']].count()

fl_date               110661
mkt_unique_carrier    110661
mkt_carrier_fl_num    110661
op_unique_carrier     110661
tail_num              110661
op_carrier_fl_num     110661
origin                110661
dest                  110661
crs_dep_time          110661
crs_arr_time          110661
crs_elapsed_time      110661
flights               110661
distance              110661
arr_delay             110661
carrier_delay          22289
weather_delay          22289
dep_delay             110661
taxi_out              110661
taxi_in               110661
dtype: int64

__c) *mkt carrier with high incidence of arr_delay*__

In [125]:
delayed_mkt_arr= df_flight.groupby(['mkt_unique_carrier']).arr_delay.mean().round(0).sort_values(ascending=False)
delayed_mkt_arr.head()

mkt_unique_carrier
B6    12.0
UA    11.0
F9     9.0
G4     9.0
AA     6.0
Name: arr_delay, dtype: float64

In [126]:
delayed_mkt_arr= delayed_mkt_arr.reset_index()

In [127]:
delayed_mkt_arr

,mkt_unique_carrier,arr_delay
0,B6,12.0
1,UA,11.0
2,F9,9.0
3,G4,9.0
4,AA,6.0
5,NK,5.0
6,DL,3.0
7,WN,2.0
8,AS,1.0
9,HA,1.0


In [128]:
delayed_mkt_arr.columns= ['mkt_unique_carrier','mkt_arr_delay']

### op carrier
__*a) op carrier with high incidence of carrier_delay*__

In [129]:
delayed_op_carrier=df_flight.groupby(['op_unique_carrier']).carrier_delay.mean().round(0).sort_values(ascending=False)
delayed_op_carrier= delayed_op_carrier.reset_index()

In [130]:
delayed_op_carrier.head()

,op_unique_carrier,carrier_delay
0,OO,30.0
1,B6,28.0
2,CP,28.0
3,YV,27.0
4,EV,27.0


In [131]:
delayed_op_carrier.columns= ['op_unique_carrier','op_carrier_delay']

In [132]:
delayed_op_carrier.head()

,op_unique_carrier,op_carrier_delay
0,OO,30.0
1,B6,28.0
2,CP,28.0
3,YV,27.0
4,EV,27.0


__*b) op_unique_carrier with high incidence of departure delay*__

In [133]:
departure_delay_op_carrier=df_flight.groupby(['op_unique_carrier']).dep_delay.mean().round(0).sort_values(ascending=False)
departure_delay_op_carrier= departure_delay_op_carrier.reset_index()

In [134]:
departure_delay_op_carrier.head()

,op_unique_carrier,dep_delay
0,C5,26.0
1,KS,22.0
2,EV,20.0
3,B6,18.0
4,AX,17.0


In [135]:
departure_delay_op_carrier.columns= ['op_unique_carrier','op_carrier_dep_delay']

In [136]:
departure_delay_op_carrier.head()

,op_unique_carrier,op_carrier_dep_delay
0,C5,26.0
1,KS,22.0
2,EV,20.0
3,B6,18.0
4,AX,17.0


__*c) op carrier with high incidence of arr_delay*__

In [137]:
delayed_op_arr=df_flight.groupby(['op_unique_carrier']).arr_delay.mean().round(0).sort_values(ascending=False)
delayed_op_arr= delayed_op_arr.reset_index()

In [138]:
delayed_op_arr.head()

,op_unique_carrier,arr_delay
0,C5,24.0
1,KS,21.0
2,EV,19.0
3,AX,16.0
4,B6,12.0


In [139]:
delayed_op_arr.columns= ['op_unique_carrier','op_arr_delay']

In [140]:
delayed_op_arr.head()

,op_unique_carrier,op_arr_delay
0,C5,24.0
1,KS,21.0
2,EV,19.0
3,AX,16.0
4,B6,12.0


__Relation between 'mkt_carrier_fl_num' and 'op_carrier_fl_num'__

In [141]:
df_flight[df_flight['mkt_carrier_fl_num'] != df_flight['op_carrier_fl_num']][['mkt_carrier_fl_num', 'op_carrier_fl_num']].count()

mkt_carrier_fl_num    30
op_carrier_fl_num     30
dtype: int64

- We have only 30 row that has the difference between 'mkt_carrier_fl_num' and 'op_carrier_fl_num'
=> we can assume that 'mkt_carrier_fl_num' and 'op_carrier_fl_num' is the same thing
=> keep the feature **'mkt_carrier_fl_num'**

In [142]:
df_flight = df_flight.drop(columns=['op_carrier_fl_num'])

In [143]:
df_flight_test = df_flight_test.drop(columns=['op_carrier_fl_num'])

In [144]:
df_flight.shape

(293446, 18)

In [145]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293446 entries, 0 to 299999
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fl_date             293446 non-null  datetime64[ns]
 1   mkt_unique_carrier  293446 non-null  object        
 2   mkt_carrier_fl_num  293446 non-null  int64         
 3   op_unique_carrier   293446 non-null  object        
 4   tail_num            293446 non-null  object        
 5   origin              293446 non-null  object        
 6   dest                293446 non-null  object        
 7   crs_dep_time        293446 non-null  int64         
 8   crs_arr_time        293446 non-null  int64         
 9   crs_elapsed_time    293446 non-null  float64       
 10  flights             293446 non-null  float64       
 11  distance            293446 non-null  float64       
 12  arr_delay           293446 non-null  float64       
 13  carrier_delay       56793 non

In [146]:
# Flights has only 1 value = 1
df_flight['flights'].unique()

array([1.])

In [147]:
# Drop the flights column
df_flight = df_flight.drop(columns=['flights'])

In [148]:
df_flight_test = df_flight_test.drop(columns=['flights'])

In [149]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293446 entries, 0 to 299999
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fl_date             293446 non-null  datetime64[ns]
 1   mkt_unique_carrier  293446 non-null  object        
 2   mkt_carrier_fl_num  293446 non-null  int64         
 3   op_unique_carrier   293446 non-null  object        
 4   tail_num            293446 non-null  object        
 5   origin              293446 non-null  object        
 6   dest                293446 non-null  object        
 7   crs_dep_time        293446 non-null  int64         
 8   crs_arr_time        293446 non-null  int64         
 9   crs_elapsed_time    293446 non-null  float64       
 10  distance            293446 non-null  float64       
 11  arr_delay           293446 non-null  float64       
 12  carrier_delay       56793 non-null   float64       
 13  weather_delay       56793 non

In [150]:
# We dont use the carrier_delay, weather_delay and dep_delay anymore
df_flight = df_flight.drop(columns=['carrier_delay','weather_delay', 
                                    'dep_delay', 'taxi_out', 'taxi_in'])

In [151]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293446 entries, 0 to 299999
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fl_date             293446 non-null  datetime64[ns]
 1   mkt_unique_carrier  293446 non-null  object        
 2   mkt_carrier_fl_num  293446 non-null  int64         
 3   op_unique_carrier   293446 non-null  object        
 4   tail_num            293446 non-null  object        
 5   origin              293446 non-null  object        
 6   dest                293446 non-null  object        
 7   crs_dep_time        293446 non-null  int64         
 8   crs_arr_time        293446 non-null  int64         
 9   crs_elapsed_time    293446 non-null  float64       
 10  distance            293446 non-null  float64       
 11  arr_delay           293446 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 29.1+ MB


In [152]:
df_flight_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150623 entries, 0 to 150622
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Unnamed: 0          150623 non-null  int64         
 1   fl_date             150623 non-null  datetime64[ns]
 2   mkt_unique_carrier  150623 non-null  object        
 3   mkt_carrier_fl_num  150623 non-null  int64         
 4   op_unique_carrier   150623 non-null  object        
 5   tail_num            150499 non-null  object        
 6   origin              150623 non-null  object        
 7   dest                150623 non-null  object        
 8   crs_dep_time        150623 non-null  int64         
 9   crs_arr_time        150623 non-null  int64         
 10  crs_elapsed_time    150623 non-null  int64         
 11  distance            150623 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 13.8+ MB


In [153]:
df_flight_test = df_flight_test.iloc[:,1:]

In [154]:
df_flight_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150623 entries, 0 to 150622
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fl_date             150623 non-null  datetime64[ns]
 1   mkt_unique_carrier  150623 non-null  object        
 2   mkt_carrier_fl_num  150623 non-null  int64         
 3   op_unique_carrier   150623 non-null  object        
 4   tail_num            150499 non-null  object        
 5   origin              150623 non-null  object        
 6   dest                150623 non-null  object        
 7   crs_dep_time        150623 non-null  int64         
 8   crs_arr_time        150623 non-null  int64         
 9   crs_elapsed_time    150623 non-null  int64         
 10  distance            150623 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 12.6+ MB


### Feature Engineering

__1. create new features of day of week and month for fl_date__

In [155]:
# extract the weekday and month from fl_date
df_flight['fl_month'] = df_flight['fl_date'].dt.month
df_flight['fl_wday'] = df_flight['fl_date'].dt.weekday

In [156]:
df_flight_test['fl_month'] = df_flight_test['fl_date'].dt.month
df_flight_test['fl_wday'] = df_flight_test['fl_date'].dt.weekday

In [157]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3


__2. mkt_unique_carrier >> feature: mkt_carrier_delay, mkt_carrier_dep_delay, mkt_arr_delay (Ordinal encoder)__

In [158]:
#carrier_delay, carrier with high incidence of carrier delay
df_flight = df_flight.merge(delayed_mkt_carrier,how='left')

In [159]:
df_flight_test = df_flight_test.merge(delayed_mkt_carrier,how='left')

In [160]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday,mkt_carrier_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0,25.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6,17.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6,21.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0,22.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3,25.0


In [161]:
#dep_delay, carrier with high incidence of dep_delay
df_flight= df_flight.merge(departure_delay_mkt_carrier,how='left')

In [162]:
df_flight_test= df_flight_test.merge(departure_delay_mkt_carrier,how='left')

In [163]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0,25.0,10.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6,17.0,10.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6,21.0,10.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0,22.0,15.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3,25.0,10.0


In [164]:
df_flight = df_flight.merge(delayed_mkt_arr,how='left')

In [165]:
df_flight_test = df_flight_test.merge(delayed_mkt_arr,how='left')

In [166]:
df_flight[['mkt_carrier_delay','mkt_carrier_dep_delay', 'mkt_arr_delay']].isna().sum()

mkt_carrier_delay        0
mkt_carrier_dep_delay    0
mkt_arr_delay            0
dtype: int64

__4. op_unique_carrier >> feature: op_carrier_delay, op_carrier_dep_delay (Ordinal encoder)__

In [167]:
#op_carrier_delay, op_carrier with high incidence of carrier delay
df_flight = df_flight.merge(delayed_op_carrier,how='left')

In [168]:
df_flight_test = df_flight_test.merge(delayed_op_carrier,how='left')

In [169]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0,25.0,10.0,3.0,24.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6,17.0,10.0,2.0,17.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6,21.0,10.0,6.0,24.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0,22.0,15.0,11.0,14.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3,25.0,10.0,3.0,24.0


In [170]:
#op_carrier_delay, op_carrier with high incidence of carrier delay
df_flight = df_flight.merge(delayed_op_carrier,how='left')

In [171]:
df_flight_test = df_flight_test.merge(delayed_op_carrier,how='left')

In [172]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0,25.0,10.0,3.0,24.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6,17.0,10.0,2.0,17.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6,21.0,10.0,6.0,24.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0,22.0,15.0,11.0,14.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3,25.0,10.0,3.0,24.0


In [173]:
#op_carrier_delay, op_carrier with high incidence of departure delay
df_flight = df_flight.merge(departure_delay_op_carrier,how='left')

In [174]:
df_flight_test = df_flight_test.merge(departure_delay_op_carrier,how='left')

In [175]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay,op_carrier_dep_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0,25.0,10.0,3.0,24.0,8.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6,17.0,10.0,2.0,17.0,10.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6,21.0,10.0,6.0,24.0,12.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0,22.0,15.0,11.0,14.0,8.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3,25.0,10.0,3.0,24.0,8.0


In [176]:
df_flight[['op_carrier_delay','op_carrier_dep_delay']].isna().sum()

op_carrier_delay        0
op_carrier_dep_delay    0
dtype: int64

**5. mkt_carrier_fl_num >> Feature: mkt_carrier_fl_num_weather_delay, mkt_carrier_fl_num_dep_delay**

In [177]:
df_flight= df_flight.merge(bad_delay_airport,how='left')
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,fl_month,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay,op_carrier_dep_delay,mkt_carrier_fl_num_arr_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,1124.0,-11.0,1,0,25.0,10.0,3.0,24.0,8.0,2.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,1788.0,12.0,1,6,17.0,10.0,2.0,17.0,10.0,9.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,666.0,-15.0,1,6,21.0,10.0,6.0,24.0,12.0,-3.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,209.0,-17.0,1,0,22.0,15.0,11.0,14.0,8.0,5.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,1931.0,-13.0,1,3,25.0,10.0,3.0,24.0,8.0,2.0


In [178]:
df_flight= df_flight.merge(bad_delay_dest,how='left')

In [179]:
df_flight_test= df_flight_test.merge(bad_delay_airport,how='left')

In [180]:
df_flight_test= df_flight_test.merge(bad_delay_dest,how='left')

In [181]:
df_flight= df_flight.merge(taxi_out_airport,how='left')
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,fl_month,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay,op_carrier_dep_delay,mkt_carrier_fl_num_arr_delay,dest_arr_delay,mkt_carrier_fl_num_taxi_out
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,...,1,0,25.0,10.0,3.0,24.0,8.0,2.0,1.0,16.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,...,1,6,17.0,10.0,2.0,17.0,10.0,9.0,5.0,20.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,...,1,6,21.0,10.0,6.0,24.0,12.0,-3.0,1.0,15.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,...,1,0,22.0,15.0,11.0,14.0,8.0,5.0,21.0,18.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,...,1,3,25.0,10.0,3.0,24.0,8.0,2.0,6.0,18.0


In [182]:
df_flight_test= df_flight_test.merge(taxi_out_airport,how='left')

In [183]:
df_flight= df_flight.merge(taxi_in_airport,how='left')
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,fl_wday,mkt_carrier_delay,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay,op_carrier_dep_delay,mkt_carrier_fl_num_arr_delay,dest_arr_delay,mkt_carrier_fl_num_taxi_out,mkt_carrier_fl_num_taxi_in
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,...,0,25.0,10.0,3.0,24.0,8.0,2.0,1.0,16.0,8.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,...,6,17.0,10.0,2.0,17.0,10.0,9.0,5.0,20.0,12.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,...,6,21.0,10.0,6.0,24.0,12.0,-3.0,1.0,15.0,6.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,...,0,22.0,15.0,11.0,14.0,8.0,5.0,21.0,18.0,7.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,...,3,25.0,10.0,3.0,24.0,8.0,2.0,6.0,18.0,8.0


In [184]:
df_flight= df_flight.merge(taxi_in_dest,how='left')

In [185]:
df_flight_test = df_flight_test.merge(taxi_in_airport,how='left')

In [186]:
df_flight_test = df_flight_test.merge(taxi_in_dest,how='left')

In [187]:
# Ordinal encoder by weather_delay (origin airport with high incidence of weather delay)
df_flight= df_flight.merge(bad_weather_airport,how='left')
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,mkt_carrier_dep_delay,mkt_arr_delay,op_carrier_delay,op_carrier_dep_delay,mkt_carrier_fl_num_arr_delay,dest_arr_delay,mkt_carrier_fl_num_taxi_out,mkt_carrier_fl_num_taxi_in,dest_taxi_in,mkt_carrier_fl_num_weather_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,...,10.0,3.0,24.0,8.0,2.0,1.0,16.0,8.0,6.0,1.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,...,10.0,2.0,17.0,10.0,9.0,5.0,20.0,12.0,8.0,0.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,...,10.0,6.0,24.0,12.0,-3.0,1.0,15.0,6.0,9.0,0.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,...,15.0,11.0,14.0,8.0,5.0,21.0,18.0,7.0,11.0,0.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,...,10.0,3.0,24.0,8.0,2.0,6.0,18.0,8.0,9.0,0.0


In [188]:
df_flight= df_flight.merge(bad_weather_dest,how='left')

In [189]:
df_flight_test= df_flight_test.merge(bad_weather_airport,how='left')

In [190]:
df_flight_test= df_flight_test.merge(bad_weather_dest,how='left')

In [191]:
# Ordinal encoder by departure_delay (origin airport with high incidence of departure delay)
df_flight= df_flight.merge(departure_delay_airport,how='left')
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,op_carrier_delay,op_carrier_dep_delay,mkt_carrier_fl_num_arr_delay,dest_arr_delay,mkt_carrier_fl_num_taxi_out,mkt_carrier_fl_num_taxi_in,dest_taxi_in,mkt_carrier_fl_num_weather_delay,dest_weather_delay,mkt_carrier_fl_num_dep_delay
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,...,24.0,8.0,2.0,1.0,16.0,8.0,6.0,1.0,10.0,7.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,...,17.0,10.0,9.0,5.0,20.0,12.0,8.0,0.0,2.0,14.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,...,24.0,12.0,-3.0,1.0,15.0,6.0,9.0,0.0,4.0,5.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,...,14.0,8.0,5.0,21.0,18.0,7.0,11.0,0.0,4.0,9.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,...,24.0,8.0,2.0,6.0,18.0,8.0,9.0,0.0,2.0,6.0


In [192]:
df_flight= df_flight.merge(departure_delay_dest,how='left')

In [193]:
df_flight_test= df_flight_test.merge(departure_delay_airport,how='left')

In [194]:
df_flight_test= df_flight_test.merge(departure_delay_dest,how='left')

**6. traffic by airport by hour**

In [195]:
# round up dep_time to hour
df_flight['crs_dep_hour']= np.floor(df_flight.crs_dep_time/100)
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,mkt_carrier_fl_num_arr_delay,dest_arr_delay,mkt_carrier_fl_num_taxi_out,mkt_carrier_fl_num_taxi_in,dest_taxi_in,mkt_carrier_fl_num_weather_delay,dest_weather_delay,mkt_carrier_fl_num_dep_delay,dest_dep_delay,crs_dep_hour
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,...,2.0,1.0,16.0,8.0,6.0,1.0,10.0,7.0,8.0,16.0
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,...,9.0,5.0,20.0,12.0,8.0,0.0,2.0,14.0,10.0,16.0
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,...,-3.0,1.0,15.0,6.0,9.0,0.0,4.0,5.0,7.0,12.0
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,...,5.0,21.0,18.0,7.0,11.0,0.0,4.0,9.0,25.0,11.0
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,...,2.0,6.0,18.0,8.0,9.0,0.0,2.0,6.0,12.0,17.0


In [196]:
df_flight_test['crs_dep_hour']= np.floor(df_flight_test.crs_dep_time/100)

In [197]:
# at the departure time, how many flights are schedule to departure within the same timeslot (hrs) at the origin airport
df_flight.groupby(['origin','fl_date','crs_dep_hour'])['mkt_unique_carrier'].count().sort_values(ascending=False)

origin  fl_date     crs_dep_hour
ORD     2019-09-30  8.0             11
        2019-02-05  11.0            11
ATL     2019-09-22  9.0             10
CLT     2019-11-22  16.0            10
DEN     2019-09-30  19.0            10
                                    ..
HOU     2019-08-19  6.0              1
                    11.0             1
        2019-08-20  6.0              1
                    9.0              1
YUM     2019-12-29  21.0             1
Name: mkt_unique_carrier, Length: 205222, dtype: int64

In [198]:
df_flight['departure_traffic']=df_flight.groupby(['origin','fl_date','crs_dep_hour'])['mkt_unique_carrier'].transform('count')

In [199]:
df_flight_test['departure_traffic']=df_flight_test.groupby(['origin','fl_date','crs_dep_hour'])['mkt_unique_carrier'].transform('count')

In [200]:
df_flight.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,dest_arr_delay,mkt_carrier_fl_num_taxi_out,mkt_carrier_fl_num_taxi_in,dest_taxi_in,mkt_carrier_fl_num_weather_delay,dest_weather_delay,mkt_carrier_fl_num_dep_delay,dest_dep_delay,crs_dep_hour,departure_traffic
0,2019-01-07,DL,2185,DL,N378DA,BOS,MSP,1629,1858,209.0,...,1.0,16.0,8.0,6.0,1.0,10.0,7.0,8.0,16.0,4
1,2019-01-27,WN,5073,WN,N8719Q,TPA,PHX,1635,1915,280.0,...,5.0,20.0,12.0,8.0,0.0,2.0,14.0,10.0,16.0,1
2,2019-01-27,AA,1684,AA,N702UW,PHL,ATL,1235,1453,138.0,...,1.0,15.0,6.0,9.0,0.0,4.0,5.0,7.0,12.0,3
3,2019-01-28,UA,3592,YX,N642RW,MHT,EWR,1125,1259,94.0,...,21.0,18.0,7.0,11.0,0.0,4.0,9.0,25.0,11.0,1
4,2019-01-10,DL,1486,DL,N593NW,SLC,MCO,1730,2340,250.0,...,6.0,18.0,8.0,9.0,0.0,2.0,6.0,12.0,17.0,1


## Modeling

In [201]:
from sklearn.model_selection import train_test_split

In [202]:
X = df_flight.drop(columns=['arr_delay'])

In [203]:
Y = df_flight['arr_delay']

In [204]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293446 entries, 0 to 293445
Data columns (total 29 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   fl_date                           293446 non-null  datetime64[ns]
 1   mkt_unique_carrier                293446 non-null  object        
 2   mkt_carrier_fl_num                293446 non-null  int64         
 3   op_unique_carrier                 293446 non-null  object        
 4   tail_num                          293446 non-null  object        
 5   origin                            293446 non-null  object        
 6   dest                              293446 non-null  object        
 7   crs_dep_time                      293446 non-null  int64         
 8   crs_arr_time                      293446 non-null  int64         
 9   crs_elapsed_time                  293446 non-null  float64       
 10  distance                        

In [205]:
X = X.drop(columns=['fl_date', 'mkt_unique_carrier', 'op_unique_carrier',
                    'tail_num', 'mkt_carrier_fl_num', 'origin', 'dest',
                    'departure_traffic'])

In [142]:
# day = pd.get_dummies(X['fl_wday'], drop_first=True)
# month = pd.get_dummies(X['fl_month'], prefix='M', drop_first=True)

In [143]:
# X = X.merge(day, left_index=True, right_index=True)
# X = X.merge(month, left_index=True, right_index=True)

In [144]:
# X = X.drop(columns=['fl_wday', 'fl_month'])

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,
                                                   test_size=0.2,
                                                   random_state=42)

### Linear Regression

In [207]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234756 entries, 78882 to 121958
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   crs_dep_time                      234756 non-null  int64  
 1   crs_arr_time                      234756 non-null  int64  
 2   crs_elapsed_time                  234756 non-null  float64
 3   distance                          234756 non-null  float64
 4   fl_month                          234756 non-null  int64  
 5   fl_wday                           234756 non-null  int64  
 6   mkt_carrier_delay                 234756 non-null  float64
 7   mkt_carrier_dep_delay             234756 non-null  float64
 8   mkt_arr_delay                     234756 non-null  float64
 9   op_carrier_delay                  234756 non-null  float64
 10  op_carrier_dep_delay              234756 non-null  float64
 11  mkt_carrier_fl_num_arr_delay      234756 non-nul

In [208]:
from sklearn.linear_model import LinearRegression

In [209]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [210]:
model_lr.score(X_train, y_train)

0.04414721474680716

In [211]:
model_lr.score(X_test, y_test)

0.03832635531238027

### Decision Tree

In [212]:
from sklearn.tree import DecisionTreeRegressor

In [213]:
model_tree = DecisionTreeRegressor(min_samples_leaf=5)
model_tree.fit(X_train, y_train)

DecisionTreeRegressor(min_samples_leaf=5)

In [214]:
model_tree.score(X_train, y_train)

0.3926040142157057

In [215]:
model_tree.score(X_test, y_test)

-0.3431869867945616

### Random Forest

In [216]:
X_train.shape

(234756, 21)

In [217]:
from sklearn.ensemble import RandomForestRegressor

__Model 1__

In [218]:
model = RandomForestRegressor(n_estimators=300, max_features=10, min_samples_leaf=5,)
model.fit(X_train, y_train)

RandomForestRegressor(max_features=10, min_samples_leaf=5, n_estimators=300)

In [219]:
model.score(X_train,y_train)

0.3733989019110572

In [220]:
model.score(X_test,y_test)

0.007466014346393046

In [221]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234756 entries, 78882 to 121958
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   crs_dep_time                      234756 non-null  int64  
 1   crs_arr_time                      234756 non-null  int64  
 2   crs_elapsed_time                  234756 non-null  float64
 3   distance                          234756 non-null  float64
 4   fl_month                          234756 non-null  int64  
 5   fl_wday                           234756 non-null  int64  
 6   mkt_carrier_delay                 234756 non-null  float64
 7   mkt_carrier_dep_delay             234756 non-null  float64
 8   mkt_arr_delay                     234756 non-null  float64
 9   op_carrier_delay                  234756 non-null  float64
 10  op_carrier_dep_delay              234756 non-null  float64
 11  mkt_carrier_fl_num_arr_delay      234756 non-nul

__Model 2__

In [222]:
model_2 = RandomForestRegressor(n_estimators=500, max_features=10, min_samples_leaf=5,)
model_2.fit(X_train, y_train)

RandomForestRegressor(max_features=10, min_samples_leaf=5, n_estimators=500)

In [223]:
model_2.score(X_train,y_train)

0.374331538676957

In [224]:
model_2.score(X_test,y_test)

0.007837079392609025

__Model 3__

In [225]:
model_3 = RandomForestRegressor(n_estimators=500, min_samples_leaf=10,)
model_3.fit(X_train, y_train)

RandomForestRegressor(min_samples_leaf=10, n_estimators=500)

In [226]:
model_3.score(X_train,y_train)

0.28084786973915377

In [227]:
model_3.score(X_test,y_test)

0.027613354389047995

### Test set

In [228]:
X_test_2020 = df_flight_test.drop(columns=['fl_date', 'mkt_unique_carrier', 'op_unique_carrier',
                               'tail_num', 'mkt_carrier_fl_num', 'origin', 'dest',
                               'departure_traffic'])

In [229]:
X_test_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150623 entries, 0 to 150622
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   crs_dep_time                      150623 non-null  int64  
 1   crs_arr_time                      150623 non-null  int64  
 2   crs_elapsed_time                  150623 non-null  int64  
 3   distance                          150623 non-null  int64  
 4   fl_month                          150623 non-null  int64  
 5   fl_wday                           150623 non-null  int64  
 6   mkt_carrier_delay                 150623 non-null  float64
 7   mkt_carrier_dep_delay             150623 non-null  float64
 8   mkt_arr_delay                     150623 non-null  float64
 9   op_carrier_delay                  150623 non-null  float64
 10  op_carrier_dep_delay              150623 non-null  float64
 11  mkt_carrier_fl_num_arr_delay      150212 non-null  f

In [231]:
X_test_2020['mkt_carrier_fl_num_arr_delay'].fillna(X_test_2020['mkt_carrier_fl_num_arr_delay'].mean(), inplace=True)
X_test_2020['mkt_carrier_fl_num_taxi_out'].fillna(X_test_2020['mkt_carrier_fl_num_taxi_out'].mean(), inplace=True)
X_test_2020['mkt_carrier_fl_num_taxi_in'].fillna(X_test_2020['mkt_carrier_fl_num_taxi_in'].mean(), inplace=True)
X_test_2020['mkt_carrier_fl_num_weather_delay'].fillna(X_test_2020['mkt_carrier_fl_num_weather_delay'].mean(), inplace=True)
X_test_2020['mkt_carrier_fl_num_dep_delay'].fillna(X_test_2020['mkt_carrier_fl_num_dep_delay'].mean(), inplace=True)

In [232]:
y_predict_2020 = model_3.predict(X_test_2020)

In [233]:
y_predict_2020.shape

(150623,)

In [234]:
result = pd.read_csv('data/flights_test.csv',
                    usecols=['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num',
                            'origin', 'dest'])

In [235]:
result['predicted_delay'] = y_predict_2020

In [236]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150623 entries, 0 to 150622
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   fl_date             150623 non-null  object 
 1   mkt_carrier         150623 non-null  object 
 2   mkt_carrier_fl_num  150623 non-null  int64  
 3   origin              150623 non-null  object 
 4   dest                150623 non-null  object 
 5   predicted_delay     150623 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 6.9+ MB


In [237]:
result.to_csv('data/result_submission_3.csv', index=False)